In [1]:
! pip install polars

In [2]:
import pandas as pd
import boto3
import matplotlib.pyplot as plt
import s3fs
import polars as pl

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
#scanning in our parquet from s3
source = "s3://alexaevafernando/combined-full.parquet"
df = pl.scan_parquet(source)

In [4]:
#find unique users
unique_users_df = df.select('user').unique().collect()

In [5]:
#creating a user dictionary to reduce size of usernames
user_id_mapping = (unique_users_df.with_columns(pl.arange(0, unique_users_df.height, eager=True).alias('user_id')))


In [6]:
#assigning each user its id in the original df

user_id_mapping_lazy = user_id_mapping.lazy()

df = df.join(user_id_mapping_lazy, on='user')

df = df.drop(['user'])
df = df.rename({"user_id":"user"})

In [7]:
df_print = df.collect()

In [10]:
df_print.select("user").unique().count()

user
u32
8589732


In [11]:
#sort by user and timestamp
sorted_df = df.sort(['user','timestamp'])

In [12]:
#use sorted dataframe to calculate time differences between each pixel placed by user
time_df_df = sorted_df.with_columns([pl.col("timestamp").diff().over("user").alias("time_diff")])

In [13]:
result = time_df_df.collect()

In [15]:
#calculate in minutes
result = result.with_columns([(pl.col('time_diff').dt.total_seconds()/60).alias("time_diff_min")])

In [16]:
client = boto3.client('s3')

In [18]:
import s3fs

In [19]:
bucket = 'alexaevafernando'

file_name = 'ready.parquet'
s3_uri = f"s3://{bucket}/{file_name}"

fs = s3fs.S3FileSystem()

with fs.open(s3_uri, 'wb')as f:
    result.write_parquet(f)